In [1]:
from collections import defaultdict
import numpy as np
import operator
import os
import pandas as pd
import pickle

import logging
logging.basicConfig(level=logging.DEBUG,
        format='%(levelname)-8s [%(lineno)d] %(message)s')

In [2]:
_, index = pickle.load(open(
    '/mnt/store/home/makrai/project/verb-tensor/depCC/sparstensr_freq_2.pkl', mode='rb'))

# [Transitive sentence similarity](http://www.cs.ox.ac.uk/activities/compdistmeaning/GS2011data.txt) (Grefenstette and Sadrzadeh, EMNLP 2011)

In [3]:
svo_sim = pd.read_csv('/mnt/permanent/Language/English/Data/verb-similarity/GS2011data.txt', sep=' ', 
                 true_values=['HIGH'], false_values=['LOW'])

In [130]:
cols_ordered = ["subject", "verb", "landmark", "object", "input", "hilo", "participant"]

In [5]:
svo_sim[cols_ordered].head()

,subject,verb,landmark,object,input,hilo,participant
0,family,provide,supply,home,4,True,participant20
1,government,provide,leave,cash,3,False,participant20
2,government,provide,supply,cash,7,True,participant20
3,man,provide,leave,money,6,True,participant20
4,man,provide,supply,money,7,True,participant20


In [119]:
svo_sim.describe(percentiles=[])

,input
count,2500.000000
mean,3.570800
std,2.193561
min,0.000000
50%,3.000000
max,7.000000


In [45]:
svo_sim.corr(method='spearman')

,input,hilo,good
input,1.000000,0.516890,0.015334
hilo,0.516890,1.000000,-0.006527
good,0.015334,-0.006527,1.000000


## Tensor

In [7]:
rank = 64

In [8]:
ktensor, fit, n_iterations, exectimes = pickle.load(open(
    '/mnt/store/home/makrai/project/verb-tensor/depCC/ktensor_freq_2_{}.pkl'.format(rank), mode='rb'))
#modes = ['nsubj', 'ROOT', 'dobj']
#ktensor.U[1] /= np.apply_along_axis(np.linalg.norm, 1, ktensor.U[1]).reshape((-1,1))


In [9]:
oov = defaultdict(int)
def verb_pred(ser):
    try:
        predicted_ids = np.argsort((-ktensor.lmbda * 
                                    ktensor.U[0][index['nsubj'][ser[0]]]) .dot(
            (ktensor.U[1] * ktensor.U[2][index['dobj'][ser[0]]]).T))#] # 
        return [index['ROOT'].inverse[i] for i in predicted_ids][:10]
    except KeyError as e:
        oov[e.args] += 1
        return []

In [10]:
%time svo_sim['predicted'] = svo_sim[['subject', 'object']].apply(verb_pred, axis=1)

CPU times: user 8min 47s, sys: 21min 17s, total: 30min 5s
Wall time: 1min 53s


In [11]:
sorted(oov.items(), key=operator.itemgetter(1), reverse=True)

[(('tribunal',), 26),
 (('runway',), 26),
 (('cinema',), 24),
 (('spokesman',), 24)]

In [145]:
def is_good(ser):
    #return ser['verb'] in ser['predicted']# or 
    return ser['landmark'] in ser['predicted']
        
svo_sim['good'] = svo_sim.apply(is_good, axis=1)

In [146]:
svo_sim.good.sum()

37

In [142]:
svo_sim.shape

(2500, 9)

In [150]:
len(svo_sim.landmark.unique())

20

In [135]:
cols_ordered = ["subject", "verb", "landmark", "object", "input", "hilo", "participant", 'predicted', 'good']

In [137]:
svo_sim[svo_sim.input>6][svo_sim.good][cols_ordered]

/home/makrai/tool/python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,subject,verb,landmark,object,input,hilo,participant,predicted,good
7,employer,provide,supply,training,7,True,participant20,"[need, include, tell, provide, know, see, offe...",True
64,student,meet,satisfy,requirement,7,True,participant20,"[tell, ask, invite, meet, encourage, urge, tea...",True
164,student,meet,satisfy,requirement,7,True,participant21,"[tell, ask, invite, meet, encourage, urge, tea...",True
207,employer,provide,supply,training,7,True,participant22,"[need, include, tell, provide, know, see, offe...",True
308,employer,provide,supply,training,7,True,participant23,"[need, include, tell, provide, know, see, offe...",True
318,company,provide,supply,service,7,True,participant23,"[have, use, need, want, buy, own, offer, provi...",True
320,system,provide,supply,facility,7,True,participant23,"[have, offer, provide, allow, create, require,...",True
381,poll,show,express,support,7,True,participant23,"[tell, see, take, get, have, show, find, put, ...",True
508,employer,provide,supply,training,7,True,participant25,"[need, include, tell, provide, know, see, offe...",True
520,system,provide,supply,facility,7,True,participant25,"[have, offer, provide, allow, create, require,...",True


In [122]:
len(svo_sim[svo_sim.input>3]['landmark'].unique())

20

In [123]:
svo_sim[svo_sim.input>3].groupby('good').size()

good
False    1045
True      133
dtype: int64

In [112]:
svo_sim.groupby('good').size()

good
False    2228
True      272
dtype: int64

In [33]:
svo_sim[svo_sim.good][cols_ordered]#.groupby('verb').size()

,subject,verb,landmark,object,input,hilo,participant,predicted,predicted,predicted
7,employer,provide,supply,training,7,True,participant20,"[need, include, tell, provide, know, see, offe...","[need, include, tell, provide, know, see, offe...","[need, include, tell, provide, know, see, offe..."
8,family,provide,leave,home,1,False,participant20,"[have, need, tell, want, bring, leave, mean, b...","[have, need, tell, want, bring, leave, mean, b...","[have, need, tell, want, bring, leave, mean, b..."
24,child,meet,satisfy,house,1,False,participant20,"[have, need, love, want, tell, bring, know, as...","[have, need, love, want, tell, bring, know, as...","[have, need, love, want, tell, bring, know, as..."
57,student,meet,visit,requirement,1,False,participant20,"[tell, ask, invite, meet, encourage, urge, tea...","[tell, ask, invite, meet, encourage, urge, tea...","[tell, ask, invite, meet, encourage, urge, tea..."
64,student,meet,satisfy,requirement,7,True,participant20,"[tell, ask, invite, meet, encourage, urge, tea...","[tell, ask, invite, meet, encourage, urge, tea...","[tell, ask, invite, meet, encourage, urge, tea..."
75,employer,provide,leave,training,1,False,participant20,"[need, include, tell, provide, know, see, offe...","[need, include, tell, provide, know, see, offe...","[need, include, tell, provide, know, see, offe..."
78,child,meet,visit,house,4,True,participant20,"[have, need, love, want, tell, bring, know, as...","[have, need, love, want, tell, bring, know, as...","[have, need, love, want, tell, bring, know, as..."
79,boy,meet,satisfy,girl,2,False,participant20,"[need, use, want, tell, meet, call, ask, be, p...","[need, use, want, tell, meet, call, ask, be, p...","[need, use, want, tell, meet, call, ask, be, p..."
88,boy,meet,visit,girl,6,True,participant20,"[need, use, want, tell, meet, call, ask, be, p...","[need, use, want, tell, meet, call, ask, be, p...","[need, use, want, tell, meet, call, ask, be, p..."
107,employer,provide,supply,training,6,True,participant21,"[need, include, tell, provide, know, see, offe...","[need, include, tell, provide, know, see, offe...","[need, include, tell, provide, know, see, offe..."


In [29]:
svo_sim[cols_ordered].sample()

,subject,verb,landmark,object,input,hilo,participant,predicted
216,scholar,write,publish,book,6,True,participant22,"[need, find, see, spend, meet, choose, go, ord..."


# [Verb prediction (tensor)](https://everest.hds.utc.fr/doku.php?id=en:lfmnips12) (Jenatton+ NIPS 2012)

In [13]:
dataset_dir = '/mnt/permanent/Language/English/Data/verb-similarity/SVO-tensor-dataset/'
def get_index(pos):
    with open(os.path.join(dataset_dir, 'svo-{}s.lst'.format(pos))) as infile:
        return {i+1: ' '.join(line.strip().split('_')[2:-1]) for i, line in enumerate(infile)}

In [14]:
svo_df = pd.read_csv(os.path.join(dataset_dir, 'svo_data_train_1000000.dat'), sep='\t', header=None, 
                     names=['subject', 'verb', 'object'])

In [15]:
index_verb = get_index('verb')
index_noun = get_index('noun')

In [16]:
svo_df.subject = svo_df.subject.apply(index_noun.get)
svo_df.verb = svo_df.verb.apply(index_verb.get)
svo_df.object = svo_df.object.apply(index_noun.get)

In [17]:
svo_df.head(10)

,subject,verb,object
0,man,swipe,credit card
1,george westinghouse,illuminate,exposition
2,personality,vanish,moment
3,fable,highlight,role
4,secretion,call,surfactant
5,device,store,multimedia
6,diversity,base,engineering
7,troop,destroy,weather station
8,flute,include,vein
9,complication,relate,emphysema


# [SimVerb](http://people.ds.cam.ac.uk/dsg40/simverb.html) (Gerz+ EMNLP 2016)

In [18]:
simverb = pd.read_csv('/mnt/permanent/Language/English/Data/verb-similarity/simverb-3500/SimVerb-3500.txt', sep='\t',
                      header=None, names=['verb1', 'verb2', 'pos', 'sim', 'rel'])

In [19]:
simverb.head()

,verb1,verb2,pos,sim,rel
0,take,remove,V,6.81,SYNONYMS
1,walk,trail,V,4.81,COHYPONYMS
2,feed,starve,V,1.49,ANTONYMS
3,shine,polish,V,7.80,SYNONYMS
4,calculate,add,V,5.98,HYPER/HYPONYMS


In [20]:
simverb.shape

(3500, 5)

In [21]:
simverb.groupby('rel').sim.describe(percentiles=[]).sort_values('count', ascending=False)

,count,mean,std,min,50%,max
rel,,,,,,
NONE,2093.0,3.431276,2.342695,0.0,3.150,9.79
HYPER/HYPONYMS,800.0,6.012525,2.104537,0.5,6.310,9.96
SYNONYMS,306.0,6.789150,2.104490,0.5,7.140,9.96
COHYPONYMS,190.0,4.435526,2.381992,0.0,4.665,9.30
ANTONYMS,111.0,0.977748,1.074232,0.0,0.660,6.04


## Tensor

In [75]:
_, index = pickle.load(open(
    '/mnt/store/home/makrai/project/verb-tensor/depCC/sparstensr_freq_2.pkl', mode='rb'))

In [100]:
def tensor_based_sim(rank):
    ktensor, fit, n_iterations, exectimes = pickle.load(open(
        '/mnt/store/home/makrai/project/verb-tensor/depCC/ktensor_freq_2_{}.pkl'.format(rank), mode='rb'))
    #modes = ['nsubj', 'ROOT', 'dobj']
    ktensor.U[1] /= np.apply_along_axis(np.linalg.norm, 1, ktensor.U[1]).reshape((-1,1))
    oov = defaultdict(int)
    def pointwise_prod_more(ser):
        try:
            #return ktensor.lmbda .dot( 
            return ktensor.U[1][index['ROOT'][ser[0]]] .dot( ktensor.U[1][index['ROOT'][ser[1]]])
        except KeyError as e:
            oov[e.args] += 1
            return np.nan
    simverb['tensor_sim_{}'.format(rank)] = simverb[['verb1', 'verb2']].apply(pointwise_prod_more, axis=1)

In [101]:
sorted(oov.items(), key=operator.itemgetter(1), reverse=True)

[(('tribunal',), 26),
 (('runway',), 26),
 (('cinema',), 24),
 (('spokesman',), 24)]

In [102]:
for exp in range(1,7):
    tensor_based_sim(rank=2**exp)

/home/makrai/tool/python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in true_divide
  """
/home/makrai/tool/python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in true_divide
  """


In [103]:
simverb.corr()

,sim,tensor_sim_2,tensor_sim_4,tensor_sim_8,tensor_sim_16,tensor_sim_32,tensor_sim_64
sim,1.000000,0.026359,0.059368,0.038123,0.113215,0.119878,0.004023
tensor_sim_2,0.026359,1.000000,0.050444,0.104587,0.140193,0.161455,0.055140
tensor_sim_4,0.059368,0.050444,1.000000,0.150715,0.188653,0.223799,0.137398
tensor_sim_8,0.038123,0.104587,0.150715,1.000000,0.468405,0.186961,0.298710
tensor_sim_16,0.113215,0.140193,0.188653,0.468405,1.000000,0.503928,0.272844
tensor_sim_32,0.119878,0.161455,0.223799,0.186961,0.503928,1.000000,0.176806
tensor_sim_64,0.004023,0.055140,0.137398,0.298710,0.272844,0.176806,1.000000


In [107]:
simverb.groupby('rel').corr().sim

rel                          
ANTONYMS        sim              1.000000
                tensor_sim_2    -0.090946
                tensor_sim_4     0.009703
                tensor_sim_8    -0.059901
                tensor_sim_16   -0.050149
                tensor_sim_32   -0.012503
                tensor_sim_64    0.030099
COHYPONYMS      sim              1.000000
                tensor_sim_2     0.016585
                tensor_sim_4    -0.035097
                tensor_sim_8     0.005121
                tensor_sim_16   -0.000799
                tensor_sim_32    0.007716
                tensor_sim_64   -0.203964
HYPER/HYPONYMS  sim              1.000000
                tensor_sim_2    -0.042509
                tensor_sim_4     0.011959
                tensor_sim_8     0.013426
                tensor_sim_16    0.029561
                tensor_sim_32    0.033667
                tensor_sim_64   -0.072185
NONE            sim              1.000000
                tensor_sim_2     0.023119
    